<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [17]</a>'.</span>

In [1]:
!pip install codecarbon # reiniciar apos instalar este pacote se você estiver em um notebook

In [2]:
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig, Trainer, TrainingArguments,
    DataCollatorForLanguageModeling, EarlyStoppingCallback,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

from peft import PeftModel
import pickle
import time

In [3]:
# ------------------
# Configurações
# ------------------
# MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
MODEL_NAME = 'Qwen/Qwen3-14B-Base'
OUTPUT_DIR = "./qlora-aderencia-classes.balanced"
SEED = 42
LABELS = ["BAIXA", "MÉDIA", "ALTA"]

torch.manual_seed(SEED)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
torch.utils.checkpoint.use_reentrant = False

# ------------------
# Prompt
# ------------------
PROMPT_TMPL = """You are a thematic relevance evaluator.
Classify how related an academic work (title and abstract) is to a strategic theme.

TITLE: {title}
KEYWORDS: {keywords}
ABSTRACT: {abstract}

Answer with a number 0, 1, or 2 for RELEVANCE LEVEL (2-HIGH, 1-MEDIUM, 0-LOW) to the strategic theme: "{category}".

GENERAL CRITERIA:
- HIGH: the topic is central to the research; strong semantic coherence.
- MEDIUM: partial/indirect or secondary relation to the topic.
- LOW: weak or tangential relation; the topic is not the main focus of the work.

ANSWER: """


In [4]:
with open("my_data.pickle", "rb") as file:
    split_train = pickle.load(file)
    split_eval = pickle.load(file)
    split_test = pickle.load(file)

In [5]:
print( len(split_train) , len(split_eval), len(split_test) )

33620 4203 4203


In [6]:


# ------------------
# Tokenizer
# ------------------
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token



# ------------------
# Função de preprocessamento
# ------------------
def preprocess(example, max_input_length=1024, max_description=512):
    processed_key_words = '\n- '.join(example["descricao_keyword"].split(';'))
    gold = str(example["modelo_nivel"]).strip().upper()
    if gold == "MEDIA":
        gold = "MÉDIA"
    assert gold in LABELS
    gold_idx = LABELS.index(gold)  # 0,1,2

    # Cria prompt
    prompt = PROMPT_TMPL.format(
        title=example["nome_producao"],
        abstract=example["descricao_abstract"][:max_description],
        keywords=processed_key_words,
        category=example["tema"]
    )

    # Tokeniza prompt
    input_enc = tok(prompt, truncation=True, max_length=max_input_length, add_special_tokens=False)
    input_ids = input_enc["input_ids"]

    # Token do próximo token (classe)
    label_id = tok(str(gold_idx), add_special_tokens=False)["input_ids"][0]

    if label_id not in [15, 16, 17]:
        print( label_id )

    # labels: -100 no prompt, token da classe no final
    labels = [-100] * len(input_ids) + [label_id]
    input_ids_full = input_ids + [label_id]
    attention_mask = [1] * len(input_ids_full)
    # labels = [label_id]
    # attention_mask = [1] * len(input_ids)

    if example["modelo_nivel"].replace('E','É') not in LABELS:
        print("ERRO:", example["modelo_nivel"], "->", gold, "(", label_id, ")")
        print("PROMPT:", prompt)
        print("INPUT_IDS:", input_ids)
        print("LABELS:", labels)
        # raise ValueError("Label inválida")

    return {
        "input_ids": input_ids_full,
        "attention_mask": attention_mask,
        "labels": labels
    }

# ------------------
# Criação do Dataset
# ------------------
train_ds = split_train.map(preprocess, batched=False, remove_columns=split_train.column_names)
eval_ds = split_eval.map(preprocess, batched=False, remove_columns=split_eval.column_names)

Map:   0%|          | 0/33620 [00:00<?, ? examples/s]

Map:   0%|          | 0/4203 [00:00<?, ? examples/s]

In [7]:
for i in range(len(train_ds)):
    if train_ds['labels'][i][-1] not in [15,16,17]:
        print(i)
        print(train_ds['labels'][i][-1])

In [8]:


# ------------------
# Modelo base com quantização 4-bit
# ------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

raw_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)
raw_model = prepare_model_for_kbit_training(raw_model)

print( 'Model loaded in ', raw_model.device)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Model loaded in  cuda:0


In [9]:
for name, module in raw_model.named_modules():
    print(name)



model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.q_norm
model.layers.0.self_attn.k_norm
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.q_norm
model.layers.1.self_attn.k_norm
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_att

In [10]:
# ------------------
# LoRA
# ------------------

# pegar nomes das últimas camadas
n_last = 4
num_total_layers = len(raw_model.model.layers)
target_modules = []
for i in range(num_total_layers - n_last, num_total_layers):
    for proj in ["q_proj", "k_proj", "v_proj", "o_proj"]:
        target_modules.append(f"model.layers.{i}.self_attn.{proj}")
    for proj in ["up_proj", "down_proj"]:  # se quiser também adaptar o MLP
        target_modules.append(f"model.layers.{i}.mlp.{proj}")
print( target_modules )

['model.layers.36.self_attn.q_proj', 'model.layers.36.self_attn.k_proj', 'model.layers.36.self_attn.v_proj', 'model.layers.36.self_attn.o_proj', 'model.layers.36.mlp.up_proj', 'model.layers.36.mlp.down_proj', 'model.layers.37.self_attn.q_proj', 'model.layers.37.self_attn.k_proj', 'model.layers.37.self_attn.v_proj', 'model.layers.37.self_attn.o_proj', 'model.layers.37.mlp.up_proj', 'model.layers.37.mlp.down_proj', 'model.layers.38.self_attn.q_proj', 'model.layers.38.self_attn.k_proj', 'model.layers.38.self_attn.v_proj', 'model.layers.38.self_attn.o_proj', 'model.layers.38.mlp.up_proj', 'model.layers.38.mlp.down_proj', 'model.layers.39.self_attn.q_proj', 'model.layers.39.self_attn.k_proj', 'model.layers.39.self_attn.v_proj', 'model.layers.39.self_attn.o_proj', 'model.layers.39.mlp.up_proj', 'model.layers.39.mlp.down_proj']


In [11]:

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules
)
model = get_peft_model(raw_model, lora_config)

In [12]:
import torch

if torch.cuda.is_available():
    print("Dispositivo:", torch.cuda.get_device_name(0))
    print("Suporta bf16?", torch.cuda.is_bf16_supported())
else:
    print("Nenhuma GPU disponível")


Dispositivo: NVIDIA RTX A5000
Suporta bf16? True


In [13]:
from collections import Counter

# pega todos os rótulos
labels = split_train["modelo_nivel"]

# conta quantos de cada
contagem = Counter(labels)

# imprime
for k, v in contagem.items():
    print(f"{k}: {v}")

class_counts = [contagem[x.replace('É','E')] for x in LABELS]
print("Contagem de rótulos:", class_counts)

MEDIA: 10369
BAIXA: 16091
ALTA: 7160
Contagem de rótulos: [16091, 10369, 7160]


In [14]:
# IDs dos tokens "0","1","2"
id_0 = tok("0", add_special_tokens=False).input_ids[0]
id_1 = tok("1", add_special_tokens=False).input_ids[0]
id_2 = tok("2", add_special_tokens=False).input_ids[0]

vocab_size = model.config.vocab_size
weights = torch.zeros(vocab_size, device=model.device)

# inverso da frequência (ajustado para só 3 classes)
class_counts = torch.tensor(class_counts, dtype=torch.float, device=model.device)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_counts)

# coloca nos índices corretos do vocab
weights[id_0] = class_weights[0]
weights[id_1] = class_weights[1]
weights[id_2] = class_weights[2]

In [15]:
import torch.nn as nn
from typing import Any, Optional, Union
from codecarbon import EmissionsTracker # para calcular emissões de CO2

# Supondo que você já tenha contado os exemplos por classe
# class_counts = [100, 300, 50]  # Exemplo para 3 classes
# class_weights = [1.0 / c for c in class_counts]  # inversamente proporcional à frequência
# class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

loss_fn = nn.CrossEntropyLoss(weight=weights)


class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,
        num_items_in_batch: Optional[torch.Tensor] = None):
        labels = inputs.get("labels")

        # print(inputs)
        
        # Forward normal
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        logits = outputs.get("logits")

        # Índice do último token válido em cada exemplo
        last_idx = inputs['attention_mask'].sum(dim=1) - 1 # [batch_size]

        # Logits do último token
        batch_size = logits.size(0)
        last_logits = logits[torch.arange(batch_size), last_idx-1, :]  # [batch_size, vocab_size]

        # print( 'input ids shape', inputs['input_ids'].shape)
        # print( 'batch size', batch_size)
        # print( 'last idx', last_idx)

        # Labels do último token
        last_labels = labels[torch.arange(batch_size), last_idx]     # [batch_size]



        # print( last_logits.shape )
        # pred_ids = torch.argmax(last_logits, dim=-1)  # shape: [batch_size]

        # # converte os IDs de volta para strings
        # pred_tokens = [tok.decode([i]) for i in pred_ids]
        # print( 'previstos', pred_ids, pred_tokens )
        # print( 'reais', last_labels )

        # decoded_texts = []
        # input_ids = inputs['input_ids']
        # for i in range(input_ids.size(0)):
        #     # pega do início até last_idx[i] (inclusive)
        #     tokens = input_ids[i, : last_idx[i]]  # +1 porque slice é exclusive
        #     decoded = tok.decode(tokens)
        #     decoded_texts.append(decoded)

        # # printar
        # for i, text in enumerate(decoded_texts):
        #     print(f"Exemplo {i}: {text}")


        # Calcula a loss
        loss = loss_fn(last_logits, last_labels)


        return (loss, outputs) if return_outputs else loss
    
def train_all():

    # ------------------
    # Data collator para Causal LM
    # ------------------
    collator = DataCollatorForSeq2Seq(tokenizer=tok, padding=True)

    # ------------------
    # TrainingArguments
    # ------------------
    args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=1,
        per_device_train_batch_size=3,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=10,
        learning_rate=2e-4,
        warmup_ratio=0.03,
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        bf16=torch.cuda.is_available(),
        optim="paged_adamw_32bit",
        seed=SEED,
        report_to="none",
        dataloader_num_workers=4,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        logging_strategy="steps",      
        log_level="info",
    )

    # ------------------
    # Trainer
    # ------------------
    trainer = WeightedTrainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tok,
        data_collator=collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )

    # ------------------
    # Treino
    # ------------------
    start_time = time.time()

    tracker_train = EmissionsTracker( output_file='balanced_fine_tuning_emissions_train_logs.csv' )
    tracker_train.start()

    trainer.train()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"\nTempo total de execução: {elapsed_time:.2f} segundos")

    emissions: float = tracker_train.stop()
    print("\n\nTotal de emissões (detalhes em balanced_fine_tuning_emissions_train_logs.csv): ",emissions)

    trainer.save_model(OUTPUT_DIR)
    tok.save_pretrained(OUTPUT_DIR)

    print("Treino finalizado. Modelo salvo em", OUTPUT_DIR)


train_all()

/tmp/ipykernel_665719/1956854887.py:104: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
Using auto half precision backend


[codecarbon WARNING @ 13:18:58] Multiple instances of codecarbon are allowed to run at the same time.


[codecarbon INFO @ 13:18:58] [setup] RAM Tracking...


[codecarbon INFO @ 13:18:58] [setup] CPU Tracking...


[codecarbon WARNING @ 13:18:59] We saw that you have a Intel(R) Core(TM) i9-14900KF but we don't know it. Please contact us.


...

Special tokens file saved in ./qlora-aderencia-classes.balanced/special_tokens_map.json


chat template saved in ./qlora-aderencia-classes.balanced/chat_template.jinja


tokenizer config file saved in ./qlora-aderencia-classes.balanced/tokenizer_config.json


Special tokens file saved in ./qlora-aderencia-classes.balanced/special_tokens_map.json


Treino finalizado. Modelo salvo em ./qlora-aderencia-classes.balanced


In [16]:
del model
del raw_model
loss_fn = None
# del trainer
torch.cuda.empty_cache()

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [17]:
checkpoint_dir= os.path.join(OUTPUT_DIR, "checkpoint-4484")  # ajuste conforme necessário
# ------------------
# Modelo base com quantização 4-bit
# ------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# Aplicar o LoRA / PEFT checkpoint
model = PeftModel.from_pretrained(
    model,
    checkpoint_dir,
    device_map="auto",           # mantém a mesma distribuição
)

tok = AutoTokenizer.from_pretrained(checkpoint_dir)

loading configuration file config.json from cache at /exp/magalylica/huggingface/hub/models--Qwen--Qwen3-14B-Base/snapshots/0b0bd3732e2c374d483664439ea334928b65f304/config.json


Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 17408,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_att

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in 8-bit or 4-bit. Pass your own torch_dtype to specify the dtype of the remaining non-linear layers or pass torch_dtype=torch.float16 to remove this warning.


loading weights file model.safetensors from cache at /exp/magalylica/huggingface/hub/models--Qwen--Qwen3-14B-Base/snapshots/0b0bd3732e2c374d483664439ea334928b65f304/model.safetensors.index.json


Instantiating Qwen3ForCausalLM model under default dtype torch.float16.


Generate config GenerationConfig {
  "bos_token_id": 151643,
  "eos_token_id": 151643
}



target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.45 GiB. GPU 0 has a total capacity of 23.67 GiB of which 1.03 GiB is free. Including non-PyTorch memory, this process has 22.16 GiB memory in use. Of the allocated memory 21.39 GiB is allocated by PyTorch, and 524.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.device

In [ ]:
# ------------------
# Predict (next token)
# ------------------
import torch
import torch.nn.functional as F

# tokens de interesse
target_tokens = [tok.encode("0", add_special_tokens=False)[0], tok.encode("1", add_special_tokens=False)[0], tok.encode("2", add_special_tokens=False)[0]]

def predict_class(prompt_text):
    # tokeniza
    tokenized = tok(prompt_text, return_tensors="pt")
    input_ids = tokenized.input_ids.cuda()
    attention_mask = tokenized.attention_mask.cuda()

    # forward no modelo (sem generate)
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # logits do último token
        last_token_logits = outputs.logits[0, -1, :]
        # softmax para probabilidades
        probs = F.softmax(last_token_logits, dim=-1)

    # pegar probabilidades apenas para os tokens 0,1,2
    target_probs = {tok.decode([t]): probs[t].item() for t in target_tokens}

    # escolher token mais provável
    pred_token = max(target_probs, key=target_probs.get)

    return pred_token, target_probs


# Exemplo
example = split_test[0]
processed_key_words = '\n- '.join(example["descricao_keyword"].split(';'))

    # Cria prompt
example_prompt = PROMPT_TMPL.format(
        title=example["nome_producao"],
        abstract=example["descricao_abstract"],
        keywords=processed_key_words,
        category=example["tema"]
    )
print("Predicted class:", predict_class(example_prompt))

In [ ]:
!pip show scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
from codecarbon import EmissionsTracker # para calcular emissões de CO2

tracker = EmissionsTracker( output_file='balanced_fine_tuning_emissions.csv' )
tracker.start()

y_true = []
y_pred = []

start_time = time.time()

# .select(range(30))

for example in tqdm(split_test, desc="Inferindo"):  # percorre todo o dataset de teste
    # ---------------------
    # Prepara prompt
    processed_key_words = '\n- '.join(example["descricao_keyword"].split(';'))
    example_prompt = PROMPT_TMPL.format(
        title=example["nome_producao"],
        abstract=example["descricao_abstract"],
        keywords=processed_key_words,
        category=example["tema"]
    )

    # Predição
    pred = predict_class(example_prompt)
    y_pred.append(["0","1","2"].index(pred[0]))

    # Rótulo real
    gold = str(example["modelo_nivel"]).strip().upper()
    if gold == "MEDIA":
        gold = "MÉDIA"
    assert gold in LABELS
    y_true.append(LABELS.index(gold))

# Marca o tempo final
end_time = time.time()

# Tempo total em segundos
elapsed_time = end_time - start_time
print(f"\nTempo total de execução: {elapsed_time:.2f} segundos")


emissions: float = tracker.stop()
print("\n\nTotal de emissões (detalhes em emissions.csv): ",emissions)

# ---------------------
# Métricas
acc = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average="macro")
f1_weighted = f1_score(y_true, y_pred, average="weighted")

print("Acurácia:", acc)
print("F1-macro:", f1_macro)
print("F1-weighted:", f1_weighted)

# Relatório detalhado (por classe)
print("\nRelatório de classificação:")
print(classification_report(y_true, y_pred, target_names=LABELS))
